In [1]:
import pandas as pd
import numpy as np
import random
import time
import datetime

**Chargement de la liste récupérée sur Deezer, et premier clean**

In [2]:
history = pd.read_json("écoutes.json")
history = history.drop(['id', "readable", 'title_short', 'title_version', 'rank',
           'explicit_lyrics', 'explicit_content_lyrics','explicit_content_cover',
           'preview','md5_image', 'type'],
          axis=1)
print(history.shape)
history.head()


(182, 6)


,title,link,duration,timestamp,artist,album
0,L'ami Caouette,https://www.deezer.com/track/508426832,186,2021-11-28 15:34:58,"{'id': 1007, 'name': 'Serge Gainsbourg', 'link...","{'id': 65037012, 'title': 'Comme un boomerang'..."
1,L'autre Finistère,https://www.deezer.com/track/355615171,284,2021-11-28 15:35:50,"{'id': 1819, 'name': 'Les Innocents', 'link': ...","{'id': 40786951, 'title': 'Fous à lier', 'link..."
2,Easy On Me,https://www.deezer.com/track/1522223672,224,2021-12-04 09:19:43,"{'id': 75798, 'name': 'Adele', 'link': 'https:...","{'id': 265655342, 'title': 'Easy On Me', 'link..."
3,Macaroni (feat. Ninho),https://www.deezer.com/track/1060920512,223,2021-12-04 09:26:02,"{'id': 455796, 'name': 'Leto', 'link': 'https:...","{'id': 169225092, 'title': '100 visages', 'lin..."
4,Ne retiens pas tes larmes,https://www.deezer.com/track/1386410882,183,2021-12-04 09:26:43,"{'id': '1765', 'name': 'Amel Bent', 'link': 'h...","{'id': '233433712', 'title': 'Sorøre', 'link':..."


**Replissage des nouvelles colonnes**

In [3]:
history['Album']   = ""
history['Artiste'] = ""

for i in history.index:
    history['Album'][i]   = history['album'][i]['title']
    history['Artiste'][i] = history['artist'][i]['name']

history['Heure']    = history['timestamp'][:,].dt.hour
history['Jour']     = history['timestamp'][:,].dt.date
history['Week_day'] = history['timestamp'][:,].dt.weekday
history['Week_day'] = history['Week_day'].\
            replace([0, 1, 2, 3, 4, 5, 6],
            ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"])

history.head()


<ipython-input-3-6a2d19441d50>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['Album'][i]   = history['album'][i]['title']
<ipython-input-3-6a2d19441d50>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['Artiste'][i] = history['artist'][i]['name']


,title,link,duration,timestamp,artist,album,Album,Artiste,Heure,Jour,Week_day
0,L'ami Caouette,https://www.deezer.com/track/508426832,186,2021-11-28 15:34:58,"{'id': 1007, 'name': 'Serge Gainsbourg', 'link...","{'id': 65037012, 'title': 'Comme un boomerang'...",Comme un boomerang,Serge Gainsbourg,15,2021-11-28,Dimanche
1,L'autre Finistère,https://www.deezer.com/track/355615171,284,2021-11-28 15:35:50,"{'id': 1819, 'name': 'Les Innocents', 'link': ...","{'id': 40786951, 'title': 'Fous à lier', 'link...",Fous à lier,Les Innocents,15,2021-11-28,Dimanche
2,Easy On Me,https://www.deezer.com/track/1522223672,224,2021-12-04 09:19:43,"{'id': 75798, 'name': 'Adele', 'link': 'https:...","{'id': 265655342, 'title': 'Easy On Me', 'link...",Easy On Me,Adele,9,2021-12-04,Samedi
3,Macaroni (feat. Ninho),https://www.deezer.com/track/1060920512,223,2021-12-04 09:26:02,"{'id': 455796, 'name': 'Leto', 'link': 'https:...","{'id': 169225092, 'title': '100 visages', 'lin...",100 visages,Leto,9,2021-12-04,Samedi
4,Ne retiens pas tes larmes,https://www.deezer.com/track/1386410882,183,2021-12-04 09:26:43,"{'id': '1765', 'name': 'Amel Bent', 'link': 'h...","{'id': '233433712', 'title': 'Sorøre', 'link':...",Sorøre,Amel Bent,9,2021-12-04,Samedi


**Clean des colonnes inutiles et remplissage des données non récupérées (à ce jour)**

In [4]:
history = history.drop(['link', 'duration', 'album', 'artist'], axis=1)

history['Genre'] = "undefined" ## Temporaire le temps de récupérer les Genres sur Deezer
history['User']  = "Deez"

history = history.rename(columns={'timestamp':"Date", 'Album':"Titre", 'title':"Song"})

history = history.reindex(columns=['User','Date','Jour','Week_day', 'Heure', 'Artiste', 'Titre', 'Song', 'Genre'])

history.head()


,User,Date,Jour,Week_day,Heure,Artiste,Titre,Song,Genre
0,Deez,2021-11-28 15:34:58,2021-11-28,Dimanche,15,Serge Gainsbourg,Comme un boomerang,L'ami Caouette,undefined
1,Deez,2021-11-28 15:35:50,2021-11-28,Dimanche,15,Les Innocents,Fous à lier,L'autre Finistère,undefined
2,Deez,2021-12-04 09:19:43,2021-12-04,Samedi,9,Adele,Easy On Me,Easy On Me,undefined
3,Deez,2021-12-04 09:26:02,2021-12-04,Samedi,9,Leto,100 visages,Macaroni (feat. Ninho),undefined
4,Deez,2021-12-04 09:26:43,2021-12-04,Samedi,9,Amel Bent,Sorøre,Ne retiens pas tes larmes,undefined


**Ecriture de l'historique dans un ficher CSV et un fichier json**

In [5]:
history.to_csv('./history_Deez.csv')
history.to_json('./history_Deez.json')